In [24]:
#se cargan las librerías necesarias
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd
from datetime import datetime

# 1. Extracción de datos de la página web de la Japan Meteorological Agency

In [25]:
#hora actual
now = datetime.now()
# formato dd/mm/YY
d1 = now.strftime("%d-%m-%Y-%H%M")

In [29]:
# ruta para guardar el df con los datos scrapeados
ruta_df_japon= str('datasets/japan_scrapped_dataset-{}.csv'.format(d1))

In [3]:
# Configura las opciones de Chrome para ejecutar en modo headless (sin interfaz gráfica)
chrome_options = Options()
chrome_options.add_argument("--headless")

# Configura el webdriver
webdriver_service = Service(ChromeDriverManager().install())

[WDM] - Downloading: 100%|██████████| 6.30M/6.30M [00:00<00:00, 11.4MB/s]


In [4]:
driver = webdriver.Chrome(service=webdriver_service, options=chrome_options)
# Accede a la página web
driver.get('https://www.data.jma.go.jp/multi/quake/index.html?lang=es')

In [5]:
# Permite que la página cargue
time.sleep(5)

# Obtiene el elemento de la tabla
table = driver.find_element(By.XPATH, '//*[@id="quakeindex_table"]')

In [ ]:
# Extrae los datos y los href de la tabla
table_data = []
rows = table.find_elements(By.TAG_NAME, "tr")

#cantidad de filas
print(len(rows))

# Espera a que se carguen las filas
wait = WebDriverWait(driver, 10)

for row in rows[1:]:
    #print(row)
    cols = row.find_elements(By.TAG_NAME, "td")
    row_data = [ele.text for ele in cols]
    hrefs = [ele.find_element(By.TAG_NAME, 'a').get_attribute('href') if ele.find_elements(By.TAG_NAME, 'a') else None for ele in cols]
    row_data.extend([ele for ele in hrefs if ele])

    # Para cada href, visita la página y extrae más información
    for href in row_data[-1:]:
        # Abre una nueva pestaña
        driver.execute_script("window.open('');")
        
        # Cambia a la nueva pestaña (suponiendo que es la última a la derecha)
        driver.switch_to.window(driver.window_handles[-1])
        
        # Visita la página del href
        driver.get(href)
        
        # Espera a que se cargue la nueva página
        wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="quakeindex_table"]/tbody/tr[2]')))

        # Extrae más información de la nueva página
        elements = driver.find_elements(By.XPATH, '//*[@id="quakeindex_table"]/tbody/tr[2]/td')
        additional_info = [ele.text for ele in elements]
        
        # Añade la nueva información a los datos de la fila
        row_data.extend(additional_info)
        print(row_data)
        
        # Cierra la pestaña actual
        driver.close()
        
        # Cambia de nuevo a la pestaña original
        driver.switch_to.window(driver.window_handles[0])
        
        wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="quakeindex_table"]')))
    #apendea todos los registros
    table_data.append(row_data)

In [35]:
# Convierte los datos de la tabla a un DataFrame
df = pd.DataFrame(table_data)

In [ ]:
# Cierra el webdriver
driver.quit()

In [36]:
df.columns #nombres de las columnas

RangeIndex(start=0, stop=12, step=1)

In [37]:
#cambiar nombres de columnas
df = df.rename(columns={
    0: 'det_fecha_hora',
    1: 'epi',
    2: 'mag',
    3: 'int_max',
    4: 'emis_fecha_hora',
    5: 'href',
    6: 'det_fecha_hora_local',
    7: 'lat',
    8: 'lon',
    9: 'mag_2',
    10: 'prof_epi',
    11: 'epi_2'
})

In [38]:
df.drop(labels=['mag_2', 'epi_2'], axis = 1, inplace=True) #se eliminan las columnas redundantes

In [39]:
# dimensiones del df
df.shape

(172, 10)

In [40]:
# muestra del df
df.head()

,det_fecha_hora,epi,mag,int_max,emis_fecha_hora,href,det_fecha_hora_local,lat,lon,prof_epi
0,2023/06/28 14:27,Sur de Prefectura de Kioto,2.2,1,2023/06/28 14:31,https://www.data.jma.go.jp/multi/quake/quake_d...,2023/06/29 02:27,35.0N,135.6E,Poco profundo
1,2023/06/28 05:22,Norte de región Soya,3.0,2,2023/06/28 05:25,https://www.data.jma.go.jp/multi/quake/quake_d...,2023/06/28 17:22,44.9N,142.1E,10km
2,2023/06/28 03:02,Norte de Prefectura de Ibaraki,3.1,1,2023/06/28 03:04,https://www.data.jma.go.jp/multi/quake/quake_d...,2023/06/28 15:02,36.4N,140.6E,60km
3,2023/06/28 02:25,Este de región Hidaka,3.4,2,2023/06/28 02:28,https://www.data.jma.go.jp/multi/quake/quake_d...,2023/06/28 14:25,42.3N,142.8E,30km
4,2023/06/28 01:27,Alta mar este de Prefectura de Chiba,4.3,1,2023/06/28 01:30,https://www.data.jma.go.jp/multi/quake/quake_d...,2023/06/28 13:27,35.7N,141.4E,10km


# 2. Preprocesamiento

#### se crea la columna "id"

In [41]:
df.href.unique() # se revisa la columna href para definir un id para cada registro

array(['https://www.data.jma.go.jp/multi/quake/quake_detail.html?eventID=20230629023106&lang=es',
       'https://www.data.jma.go.jp/multi/quake/quake_detail.html?eventID=20230628172546&lang=es',
       'https://www.data.jma.go.jp/multi/quake/quake_detail.html?eventID=20230628150453&lang=es',
       'https://www.data.jma.go.jp/multi/quake/quake_detail.html?eventID=20230628142811&lang=es',
       'https://www.data.jma.go.jp/multi/quake/quake_detail.html?eventID=20230628133032&lang=es',
       'https://www.data.jma.go.jp/multi/quake/quake_detail.html?eventID=20230628121129&lang=es',
       'https://www.data.jma.go.jp/multi/quake/quake_detail.html?eventID=20230628084532&lang=es',
       'https://www.data.jma.go.jp/multi/quake/quake_detail.html?eventID=20230628005802&lang=es',
       'https://www.data.jma.go.jp/multi/quake/quake_detail.html?eventID=20230627032720&lang=es',
       'https://www.data.jma.go.jp/multi/quake/quake_detail.html?eventID=20230626015212&lang=es',
       'https://www.

In [45]:
df['id'] = df['href'].str.extract('eventID=(\d+)&') #se extrae el ID del evento
df['id'] = df['id'].astype('Int64') #se cambia el formato a int

#### se cambia el formato de la variable profundidad del epicentro

In [48]:
# extrae los números de km y convierte a tipo float
df['prof_epi_nro'] = df['prof_epi'].str.extract('(\d+)').astype(float)

# reemplaza los NaN (que corresponden a "Poco profundo") por 1
df['prof_epi_nro'].fillna(1, inplace=True)

#### se cambia el formato de las coordenadas geográficas

In [ ]:
# cambia los nombres de las columnas originales
df.rename(columns={'lat': 'lat_0', 'lon': 'lon_0'}, inplace=True)

# extrae los números y convierte a tipo float
df['lat'] = df['lat_0'].str.extract('(\d+.\d+)').astype(float)
df['lon'] = df['lon_0'].str.extract('(\d+.\d+)').astype(float)

# cambia los valores negativos para los que están en el hemisferio sur y oeste
df['lat'] = df['lat'].where(~df['lat_0'].str.contains('S'), -df['lat'])
df['lon'] = df['lon'].where(~df['lon_0'].str.contains('O'), -df['lon'])

#### cambia formato de columnas de fecha

In [55]:
df['det_fecha_hora'] = pd.to_datetime(df['det_fecha_hora'])
df['emis_fecha_hora'] = pd.to_datetime(df['emis_fecha_hora'])
df['det_fecha_hora_local'] = pd.to_datetime(df['det_fecha_hora_local'])

In [56]:
df.head() #muestra de las primeras filas

,det_fecha_hora,epi,mag,int_max,emis_fecha_hora,href,det_fecha_hora_local,lat_0,lon_0,prof_epi,id,prof_epi_nro,lat,lon
0,2023-06-28 14:27:00,Sur de Prefectura de Kioto,2.2,1,2023-06-28 14:31:00,https://www.data.jma.go.jp/multi/quake/quake_d...,2023-06-29 02:27:00,35.0N,135.6E,Poco profundo,20230629023106,1.0,35.0,135.6
1,2023-06-28 05:22:00,Norte de región Soya,3.0,2,2023-06-28 05:25:00,https://www.data.jma.go.jp/multi/quake/quake_d...,2023-06-28 17:22:00,44.9N,142.1E,10km,20230628172546,10.0,44.9,142.1
2,2023-06-28 03:02:00,Norte de Prefectura de Ibaraki,3.1,1,2023-06-28 03:04:00,https://www.data.jma.go.jp/multi/quake/quake_d...,2023-06-28 15:02:00,36.4N,140.6E,60km,20230628150453,60.0,36.4,140.6
3,2023-06-28 02:25:00,Este de región Hidaka,3.4,2,2023-06-28 02:28:00,https://www.data.jma.go.jp/multi/quake/quake_d...,2023-06-28 14:25:00,42.3N,142.8E,30km,20230628142811,30.0,42.3,142.8
4,2023-06-28 01:27:00,Alta mar este de Prefectura de Chiba,4.3,1,2023-06-28 01:30:00,https://www.data.jma.go.jp/multi/quake/quake_d...,2023-06-28 13:27:00,35.7N,141.4E,10km,20230628133032,10.0,35.7,141.4


In [57]:
df.to_csv(ruta_df_japon, index=False) #guarda el df